# Epipolar Geometry

This notebook is a study about the epipolar geometry and point triangulation using *CoppeliaSim* as a simulator to verify results in a virtual environment. 

The goal is to verify if the data generated between math and simulation is coherent. For this:
1. Instanciate two Vision Sensors in both *CoppeliaSim*;
2. Create a 3D scene with detectable points in the space;
3. Use epipolar geometry to find point to point correspondences;
4. Triangulate points;
5. Check if triangulated position matches the ground truth position of the points. 

The goal is to make a triangulation with the least amount of reconstruction error. 

---

In [ ]:
# Importing modules...
import numpy as np
import cv2

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.camera import Camera

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

# Init client 
client = RemoteAPIClient()    # Client object 
sim = client.getObject('sim') # Simulation object

# Instanciating Camera Objects

---

In [ ]:
n_cameras = 2
camera = [] # Camera objects list

for ID in range(n_cameras):
    # Get the vision sensor handle
    vision_sensor_handle = sim.getObject(f'/VisionSensor[{ID}]')

    # Get Coppelia's 3x4 object matrix
    object_matrix = np.array(sim.getObjectMatrix(vision_sensor_handle)).reshape((3,4))

    camera.append(Camera(resolution=(720,720),
                         fov_degrees=60.0,
                         object_matrix=object_matrix))

# Camera's ID (change them to see different results)
reference = 0
auxiliary = 1

# Build Essential Matrix

---

In [ ]:
# Compute relative transformation between the camera pair
relative_transformation = np.linalg.inv(camera[auxiliary].extrinsic_matrix) @ camera[reference].extrinsic_matrix
R, t = relative_transformation[0:3, 0:3], relative_transformation[0:3 , [-1]]

t = t.flatten() # Make array unidimensional
t_ss = np.array([[    0, -t[2],  t[1]],
                 [ t[2],     0, -t[0]],
                 [-t[1],  t[0],     0]]) # Skew symmetric matrix

essential_matrix = t_ss @ R

# Build Fundamental Matrix

---

In [ ]:
fundamental_matrix = np.linalg.inv(camera[auxiliary].intrinsic_matrix).T @ essential_matrix @ np.linalg.inv(camera[reference].intrinsic_matrix)

# Setting up and Plotting the 3D Scene

For setting up the scene, there will be a set of 3D points scattered across the Mathmatical Model and *CoppeliaSim*'s environment.

---

In [ ]:
# 3D points to be projected
points_to_project = np.array([[0, 1, 0],
                              [0, 0, 1],
                              [0, 0, 0]])

# Create the Scene Viewer
scene = Viewer3D(title='Camera Poses Scene', 
                 size=5)

# Add elements to the scene
scene.add_frame(np.eye(4), 'World Frame', axis_size=0.2)
scene.add_frame(camera[reference].extrinsic_matrix, 'Reference Camera Frame', axis_size=0.2)
scene.add_frame(camera[auxiliary].extrinsic_matrix, 'Auxiliary Camera Frame', axis_size=0.2) 
scene.add_points(points_to_project, 'World Points', 'black')

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Getting CoppeliaSim's Image Feed

To detect the point cloud in *CoppeliaSim*, the following process will be followed:

1. Retrieve VisionSensor image with `sim.getVisionSensorCharImage()`;
2. Flip the image;
3. Convert image to gray scale;
4. Threshold the image to evidentiate blobs;
5. Find contours of each blob;
6. Compute centroids of each blob.

---

# Compute Epipolar Lines

---

# Match Point Correspondences

---

# Triangulate 3D Location

---